# Airport queuing system

#### We are simulating a simplified airport security system at a busy airport. 

#### Passengers arrive according to a Poisson distribution with $\lambda_1 = 50$ per minute (i.e., mean interarrival rate $\mu_1 = 0.2$ minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate $\mu_2 = 0.75$ minutes. 

#### After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).

#### We want to keep average wait times below 15 minutes, so there is a penality if a passenger waiting time is longer than 15 min. In addition, there is a cost associated with each ID/boarding-pass checker and personal scanner. We want to minimise 

## Process steps

In [18]:
# 1. Arrive at the airport security system
# 2. Get in the ID/boarding-pass check queue, and wait for a server.
# 3. Get the ID/boarding-pass checked from the server.
# 4. Get in scanners queue - the shortest
# 5. Go through the personal scanner.
# 6. Leave the airport security system area

## Setting Up the Environment

In [19]:
# ---------- Import modules -----------
import simpy
import random
import statistics
import pandas as pd
import numpy as np

In [20]:
# ------------ Set constants ---------------
num_checkers = 3 # number of boarding-pass checkers
num_scanners = 3 # number of scanners

arr_rate = 15 # arrival rate (passengers per minute)
check_rate = 0.75 # boarding-pass check rate (minutes per passenger)
min_scan = 0.5 # scanner minimum time for uniform distribution
max_scan = 1.0 # scanner maximum time for uniform distribution
run_time = 720 # run time (minutes) per simulation

In [21]:
def calculate_exceeding_time(minutes):
    if minutes <= 15:
        return 0
    else:
        exceeding_time = minutes - 15
        return exceeding_time

## Creating the Model

In [22]:
# System class

class System(object):
    def __init__(self,env):
        self.env = env
        self.checker = simpy.Resource(env,num_checkers) # define number of boarding-pass checkers
        self.scanner = [] # define a set of scanners with 1 each; needed because each has its own queue
        for i in range(num_scanners):
            self.scanner.append(simpy.Resource(env,1))

    # define boarding-pass check time (exponential)
    def check(self,passenger):
        # For some reason in python, expovariate actually uses 1 over the mean, like Poisson
        yield self.env.timeout(random.expovariate(1.0/check_rate))

    # define scan time (uniform)
    def scan(self,passenger):
        yield self.env.timeout(random.uniform(min_scan,max_scan))

# Passenger process through system

def passenger(env,name,s):

    # access global variables to be able to modify them
    global check_wait
    global scan_wait
    global sys_time
    global wait_time
    global wait_cost
    global tot_through

    time_arrive = env.now # note arrival time of passenger


    # print('%s arrives at time %.2f' % (name,timeArrive))

    # Go through boarding-pass check queue
    with s.checker.request() as request:
        # print('check queue length = %d' % len(s.checker.queue))
        yield request # request a checker
        t_in = env.now # note when passenger starts being checked
        yield env.process(s.check(name)) # call check process
        t_out = env.now # note when passenger ends being checked
        check_time.append(t_out - t_in) # calculate total time for passenger to be checked

    # Find the shortest scanner queue (note: scanners are numbered 0 through numScanners-1)
    minq = 0
    for i in range(1,num_scanners):
        if (len(s.scanner[i].queue) < len(s.scanner[minq].queue)):
            minq = i

    # print('scanner queue %d lengths = %d' % (minq,len(s.scanner[minq].queue)))

    # Go through scanner queue
    with s.scanner[minq].request() as request: # use scanner number minq (the shortest, from above)
        yield request # request the scanner
        t_in = env.now # note when passenger starts being scanned
        yield env.process(s.scan(name)) # call scan process
        t_out = env.now # note when passenger ends being scanned
        scan_time.append(t_out - t_in) # calculate total time for passenger to be scanned
          
    time_leave = env.now # note time passenger finishes
    sys_time.append(time_leave - time_arrive) # calculate total time in system for passenger
    wait_time.append(sys_time[tot_through]-check_time[tot_through]-scan_time[tot_through])
    wait_cost.append(calculate_exceeding_time(wait_time[tot_through])*0.1)
    tot_through += 1 # count another passenger who got through the system


# Passenger arrival process

def setup(env):
    i = 0
    s = System(env)
    while True: # keep doing it (until simulation ends)
        yield env.timeout(random.expovariate(arr_rate)) # find tieme until next passenger is created
        i += 1 # count one more passenger

        # send the passenger through its process
        env.process(passenger(env,'Passenger %d' % i,s)) # name the passenger "Passenger i"
        


## Run the Model

In [23]:
min_checkers = 10
max_checkers = 13
min_scanners = 10
max_scanners = 13
n_0 = 20 # number of replications

results = []

for i in range(min_checkers, max_checkers,1):
    for j in range(min_scanners, max_scanners,1):
        for k in range(n_0):
            random.seed(k)
            env = simpy.Environment()

            # initialize global variables
            tot_through = 0
            check_time = []
            scan_time = []
            sys_time = []
            wait_time = []
            wait_cost = []
            num_checkers = i
            num_scanners = j
            
            # run the simulation
            env.process(setup(env)) # start passenger arrival process
            env.run(until=run_time) # run for runTime simulated minutes

             # Calculate and store average results for this replication
            rep_results = {'checkers': num_checkers,
                   'scanners': num_scanners,
                   'total_passangers': tot_through, 
                   'toatl_time': round(sum(sys_time[1:tot_through]) / tot_through, 2),
                   'check_time': round(sum(check_time[1:tot_through]) / tot_through, 2), 
                   'scan_time': round(sum(scan_time[1:tot_through]) / tot_through, 2),
                    'wait_time': round(sum(wait_time[1:tot_through]) / tot_through, 2),
                    'wait_cost': round(sum(wait_cost[1:tot_through])/1000,2),
                    'overhead_cost' : round((num_checkers + num_scanners)*(run_time/60)*150 /1000, 2),
                    'total_cost': round((round(sum(wait_cost[1:tot_through]), 2) + (num_checkers + num_scanners)*(run_time/60)*150)/1000,2)}
            results.append(rep_results)
#             print('checkers: %d , scanners: %d , passengers %d, replication %d' % (num_checkers, num_scanners, tot_through, k+1))


In [24]:
results_df = pd.DataFrame(results)
results_df

,checkers,scanners,total_passangers,toatl_time,check_time,scan_time,wait_time,wait_cost,overhead_cost,total_cost
0,10,10,9415,48.80,0.75,0.75,47.29,31.28,36.0,67.28
1,10,10,9552,38.98,0.74,0.75,37.49,22.68,36.0,58.68
2,10,10,9488,48.14,0.75,0.75,46.64,31.22,36.0,67.22
3,10,10,9544,40.93,0.75,0.75,39.44,24.41,36.0,60.41
4,10,10,9522,40.90,0.75,0.75,39.40,24.61,36.0,60.61
...,...,...,...,...,...,...,...,...,...,...
175,12,12,10792,3.21,0.75,0.75,1.71,0.00,43.2,43.20
176,12,12,10789,3.01,0.75,0.75,1.51,0.00,43.2,43.20
177,12,12,10769,2.83,0.74,0.75,1.34,0.00,43.2,43.20
178,12,12,10855,3.50,0.75,0.75,2.00,0.00,43.2,43.20


In [25]:
res = results_df.groupby(['checkers', 'scanners'])['total_cost'].agg(["mean", "var"]).reset_index()
res

,checkers,scanners,mean,var
0,10,10,64.8045,15.932016
1,10,11,64.9005,25.829321
2,10,12,66.4030,24.968875
3,11,10,64.7590,12.580262
4,11,11,43.0580,8.205880
5,11,12,44.6915,12.048508
6,12,10,64.9755,10.472121
7,12,11,42.4085,2.058645
8,12,12,43.2000,0.000000


In [26]:
d_star = 1.0
P_star = 0.95
h_1 = 3.619

In [27]:
res['N_i'] = res.apply(lambda row: max(np.ceil((h_1**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

display(res)

,checkers,scanners,mean,var,N_i
0,10,10,64.8045,15.932016,209
1,10,11,64.9005,25.829321,339
2,10,12,66.4030,24.968875,328
3,11,10,64.7590,12.580262,165
4,11,11,43.0580,8.205880,108
5,11,12,44.6915,12.048508,158
6,12,10,64.9755,10.472121,138
7,12,11,42.4085,2.058645,27
8,12,12,43.2000,0.000000,21


In [28]:
res_stage2 = []

for i in range(res.shape[0]):
    for k in range(res['N_i'][i]-n_0):
        
        random.seed(k)
        env = simpy.Environment()

        # initialize global variables
        tot_through = 0
        check_time = []
        scan_time = []
        sys_time = []
        wait_time = []
        wait_cost = []
        num_checkers = res['checkers'][i]
        num_scanners = res['scanners'][i]

        # run the simulation
        env.process(setup(env)) # start passenger arrival process
        env.run(until=run_time) # run for runTime simulated minutes

        # Calculate and store average results for this replication
        rep_results = {'checkers': num_checkers,
                       'scanners': num_scanners,
                       'total_passangers': tot_through, 
                       'toatl_time': round(sum(sys_time[1:tot_through]) / tot_through, 2),
                       'check_time': round(sum(check_time[1:tot_through]) / tot_through, 2), 
                       'scan_time': round(sum(scan_time[1:tot_through]) / tot_through, 2),
                        'wait_time': round(sum(wait_time[1:tot_through]) / tot_through, 2),
                        'wait_cost': round(sum(wait_cost[1:tot_through])/1000,2),
                        'overhead_cost' : round((num_checkers + num_scanners)*(run_time/60)*150 /1000, 2),
                        'total_cost': round((round(sum(wait_cost[1:tot_through]), 2) + (num_checkers + num_scanners)*(run_time/60)*150)/1000,2)}
#         print('checkers: %d , scanners: %d , passengers %d, replication %d' % (num_checkers, num_scanners, tot_through, k+1))
#         print(rep_results)
        res_stage2.append(rep_results)


In [29]:
results_s2_df = pd.DataFrame(res_stage2)
res_s2 = results_s2_df.groupby(['checkers', 'scanners'])['total_cost'].agg(["mean"]).reset_index()
res_s2

,checkers,scanners,mean
0,10,10,64.494815
1,10,11,63.749687
2,10,12,65.015487
3,11,10,63.705379
4,11,11,42.204886
5,11,12,43.233768
6,12,10,64.898644
7,12,11,41.525714
8,12,12,43.200000


In [30]:
res['mean(N_i - n_0)'] = res_s2["mean"]
display(res)

,checkers,scanners,mean,var,N_i,mean(N_i - n_0)
0,10,10,64.8045,15.932016,209,64.494815
1,10,11,64.9005,25.829321,339,63.749687
2,10,12,66.4030,24.968875,328,65.015487
3,11,10,64.7590,12.580262,165,63.705379
4,11,11,43.0580,8.205880,108,42.204886
5,11,12,44.6915,12.048508,158,43.233768
6,12,10,64.9755,10.472121,138,64.898644
7,12,11,42.4085,2.058645,27,41.525714
8,12,12,43.2000,0.000000,21,43.200000


In [31]:
def var_check(row):
    if row['var'] == 0:
        val = 1
    else:
        val = (n_0/row['N_i'])*(1+row['square_root'])
    return val
        


res['square_brac'] = 1-(res['N_i']-n_0)*d_star**2/(h_1**2*res['var'])
res['square_root_arg'] = 1 - res['N_i']/n_0*res['square_brac'] 
res['square_root'] = np.sqrt(res['square_root_arg'])
res['W_i1'] = res.apply(var_check, axis = 1)
# res['W_i1'] = (n_0/res['N_i'])*(1+res['square_root'])
res['W_i2'] = 1-res['W_i1']
res['X_tilda'] = res['W_i1']*res['mean'] + res['W_i2']*res['mean(N_i - n_0)']
res = res.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(res)

,checkers,scanners,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,10,10,64.8045,15.932016,209,64.494815,0.107495,0.892505,64.528104
1,10,11,64.9005,25.829321,339,63.749687,0.069785,0.930215,63.829997
2,10,12,66.4030,24.968875,328,65.015487,0.074066,0.925934,65.118254
3,11,10,64.7590,12.580262,165,63.705379,0.133519,0.866481,63.846058
4,11,11,43.0580,8.205880,108,42.204886,0.212367,0.787633,42.386060
5,11,12,44.6915,12.048508,158,43.233768,0.138383,0.861617,43.435493
6,12,10,64.9755,10.472121,138,64.898644,0.172558,0.827442,64.911906
7,12,11,42.4085,2.058645,27,41.525714,0.757105,0.242895,42.194075
8,12,12,43.2000,0.000000,21,43.200000,1.000000,0.000000,43.200000


In [32]:
res.sort_values(by=['X_tilda'])

,checkers,scanners,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
7,12,11,42.4085,2.058645,27,41.525714,0.757105,0.242895,42.194075
4,11,11,43.0580,8.205880,108,42.204886,0.212367,0.787633,42.386060
8,12,12,43.2000,0.000000,21,43.200000,1.000000,0.000000,43.200000
5,11,12,44.6915,12.048508,158,43.233768,0.138383,0.861617,43.435493
1,10,11,64.9005,25.829321,339,63.749687,0.069785,0.930215,63.829997
3,11,10,64.7590,12.580262,165,63.705379,0.133519,0.866481,63.846058
0,10,10,64.8045,15.932016,209,64.494815,0.107495,0.892505,64.528104
6,12,10,64.9755,10.472121,138,64.898644,0.172558,0.827442,64.911906
2,10,12,66.4030,24.968875,328,65.015487,0.074066,0.925934,65.118254
